In [1]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
# from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
# from nets import *
import time, os, copy, argparse
import sklearn.metrics
import multiprocessing
# from torchsummary import summary
from matplotlib import pyplot as plt

In [2]:
HERE   = 'notebook'
ROOT   = os.getcwd().replace(HERE, "")
SOLUTION = 'cnncls/'
os.chdir(os.path.join(ROOT, SOLUTION))
!pwd

/home/ubuntu/Downloads/DentalBookObjectDetection/cnncls


In [8]:
train      = {}
validation = {}
test       = {}
train['directory']      = '../resource/caries/classification/train/'
validation['directory'] = '../resource/caries/classification/validation/'
test['directory']       = '../resource/caries/classification/test/'

In [9]:
batch = 64
epoch = 50
category = 2
# model_name = 'mobilenetv2'
# model_name = "resnet18"
# model_name = "efficientnetb0"
model_name = "densenet121"
checkpoint = "checkpoint/"

In [10]:
transform = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406],
        #                      [0.229, 0.224, 0.225])
        transforms.Normalize([0.475, 0.475, 0.475],
                             [0.165, 0.165, 0.165])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.475, 0.475, 0.475],
                             [0.165, 0.165, 0.165])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.475, 0.475, 0.475],
                             [0.165, 0.165, 0.165])
    ])    
}

In [11]:
dataset = {
    'train': datasets.ImageFolder(root=train['directory'], transform=transform['train']),
    'validation': datasets.ImageFolder(root=validation['directory'], transform=transform['validation']),
    'test': datasets.ImageFolder(root=test['directory'], transform=transform['test'])
}

In [12]:
dataset['train'].class_to_idx

{'carious': 0, 'normal_crevice': 1}

In [13]:
size = {
    'train':dataset['train'].__len__(),
    'validation':dataset['validation'].__len__(),
    'test':dataset['test'].__len__()
}

In [14]:
loader = {
    'train':data.DataLoader(dataset['train'], batch_size=batch, shuffle=True, drop_last=True),
    'validation':data.DataLoader(dataset['validation'], batch_size=batch, shuffle=False, drop_last=False),
    'test':data.DataLoader(dataset['test'], batch_size=batch, shuffle=False, drop_last=False)
}

In [15]:
device = 'cuda'

In [16]:
# m = models.efficientnet_b7(pretrained=True)
# m = list(m.children())[:-1]
# m = nn.Sequential(*m)
# x = torch.randn((16, 3, 224, 224))
# m(x).shape

In [17]:
if(model_name=='efficientnetb7'):

    backbone = models.efficientnet_b7(pretrained=True)
    feature = list(backbone.children())[:-1]
    model = nn.Sequential(*feature, nn.AdaptiveAvgPool2d(1), nn.Flatten(1, -1), nn.Linear(2560, category))
    pass

if(model_name=='efficientnetb0'):

    backbone = models.efficientnet_b0(pretrained=True)
    feature = list(backbone.children())[:-1]
    model = nn.Sequential(*feature, nn.AdaptiveAvgPool2d(1), nn.Flatten(1, -1), nn.Linear(1280, category))
    pass

if(model_name=='resnet18'):

    backbone = models.resnet18(pretrained=True)
    feature = list(backbone.children())[:-1]
    model = nn.Sequential(*feature, nn.AdaptiveAvgPool2d(1), nn.Flatten(1, -1), nn.Linear(512, category))
    pass

if(model_name=='densenet121'):

    backbone = models.densenet121(pretrained=True)
    feature = list(backbone.children())[:-1]
    model = nn.Sequential(*feature, nn.AdaptiveAvgPool2d(1), nn.Flatten(1, -1), nn.Linear(1024, category))
    pass

if(model_name=='mobilenetv2'):

    backbone = models.mobilenet_v2(pretrained=True)
    feature = list(backbone.children())[:-1]
    model = nn.Sequential(*feature, nn.AdaptiveAvgPool2d(1), nn.Flatten(1, -1), nn.Linear(1280, category))
    pass

In [18]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()

##
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, dampening=0, weight_decay=0)
# schedule = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

##
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
schedule = None

##
# optimizer = optim.Adadelta(model.parameters(), lr=0.0001, rho=0.9, eps=1e-06, weight_decay=0)
# schedule = None

In [19]:
def learn(model, criterion, optimizer, schedule, epoch):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    # writer = SummaryWriter()
    
    for e in range(epoch):

        print('Epoch {}/{}'.format(e, epoch - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for mode in ['train', 'validation', 'test']:

            if(mode == 'train'): model.train()  # Set model to training mode
            if(mode == 'validation'): model.eval()   # Set model to evaluate mode
            if(mode == 'test'): model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in loader[mode]:

                # inputs = inputs.to(device, non_blocking=True)
                # labels = labels.to(device, non_blocking=True)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                if(mode=='train'):

                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    pass

                if(mode=='validation' or mode=='test'):

                    with torch.no_grad():

                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
                        pass

                    pass

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                pass

            if(mode == 'train'): 
                
                if(schedule): schedule.step()
                pass
            
            epoch_loss = running_loss / size[mode]
            epoch_acc = running_corrects.double() / size[mode]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(mode, epoch_loss, epoch_acc))

            # deep copy the model
            if(mode == 'validation' and epoch_acc > best_acc):
                
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                pass

            pass

        pass

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    best_model = model
    best_model.load_state_dict(best_model_wts)
    return(best_model)

In [20]:
better_model = learn(model, criterion, optimizer, schedule, epoch=epoch)

Epoch 0/49
----------
train Loss: 0.3843 Acc: 0.8041
validation Loss: 0.3332 Acc: 0.8366
test Loss: 0.3450 Acc: 0.8612
Epoch 1/49
----------
train Loss: 0.2150 Acc: 0.9011
validation Loss: 0.2778 Acc: 0.8742
test Loss: 0.2596 Acc: 0.8855
Epoch 2/49
----------
train Loss: 0.1719 Acc: 0.9214
validation Loss: 0.2560 Acc: 0.8852
test Loss: 0.2808 Acc: 0.8943
Epoch 3/49
----------
train Loss: 0.1355 Acc: 0.9330
validation Loss: 0.3027 Acc: 0.8609
test Loss: 0.3029 Acc: 0.8744
Epoch 4/49
----------
train Loss: 0.1264 Acc: 0.9357
validation Loss: 0.3384 Acc: 0.8433
test Loss: 0.3171 Acc: 0.8767
Epoch 5/49
----------
train Loss: 0.1094 Acc: 0.9431
validation Loss: 0.2933 Acc: 0.8918
test Loss: 0.3220 Acc: 0.8855
Epoch 6/49
----------
train Loss: 0.0885 Acc: 0.9516
validation Loss: 0.2670 Acc: 0.8830
test Loss: 0.3124 Acc: 0.8943
Epoch 7/49
----------
train Loss: 0.0838 Acc: 0.9549
validation Loss: 0.3325 Acc: 0.8874
test Loss: 0.3458 Acc: 0.8789
Epoch 8/49
----------
train Loss: 0.0687 Acc: 0.

In [21]:
os.makedirs(checkpoint, exist_ok=True)
torch.save(better_model, "{}/{}.pt".format(checkpoint, model_name))

In [22]:
@torch.no_grad()
def inference(model, loader, device='cpu'):

    model = model.to(device)
    model.eval()
    truth      = []
    prediction = []
    score      = []
    for inputs, labels in loader:

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        truth      += labels.to('cpu').tolist()
        prediction += preds.to('cpu').tolist()
        score      += [outputs]
        continue
    
    score = torch.cat(score, 0).to('cpu').numpy()
    out = (truth, prediction, score)
    return(out)

In [23]:
mode = ['validation', 'test']
for m in mode:
    
    truth, pred, prob =  inference(better_model, loader[m], 'cpu')
    report = sklearn.metrics.classification_report(y_true=truth, y_pred=pred)
    auc = sklearn.metrics.roc_auc_score(truth, prob[:,1])
    fusetable = sklearn.metrics.confusion_matrix(y_true=truth, y_pred=pred)
    print(model_name)
    print(m)
    print(report)
    print(auc)
    print(fusetable)
    continue

densenet121
validation
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       255
           1       0.91      0.92      0.91       198

    accuracy                           0.92       453
   macro avg       0.92      0.92      0.92       453
weighted avg       0.92      0.92      0.92       453

0.9643295702119232
[[236  19]
 [ 16 182]]
densenet121
test
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       250
           1       0.91      0.85      0.88       204

    accuracy                           0.90       454
   macro avg       0.90      0.89      0.89       454
weighted avg       0.90      0.90      0.90       454

0.956862745098039
[[233  17]
 [ 30 174]]


In [24]:
print('done')

done


---